In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import time
import math
import sys
import matplotlib.pyplot as plt

sys.path.append(os.path.join('..','..'))
import EnergyPricesLibrary as Ep
import CustomMetrics

from kerastuner.tuners import BayesianOptimization
from sklearn.metrics import mean_squared_error

%load_ext autoreload
%autoreload 2

In [2]:
def build_model_concat(hourly_input_shape,daily_input_shape,n_steps_out=24):
    
    input_1 = tf.keras.layers.Input(shape=hourly_input_shape)
    input_2 = tf.keras.layers.Input(shape=daily_input_shape)

    model_1_1 = tf.keras.layers.GRU(units=448,activation='tanh',kernel_regularizer=tf.keras.regularizers.L1(l1=0.06),dropout=0.36,return_sequences=True)(input_1)
    model_1_1 = tf.keras.layers.GRU(units=128,activation='tanh',kernel_regularizer=tf.keras.regularizers.L1(l1=0.09),dropout=0.36,return_sequences=False)(model_1_1)
    model_1_1 = tf.keras.layers.Dense(units=n_steps_out,activation=None)(model_1_1)

    model_2_2 = tf.keras.layers.GRU(units=256,activation='tanh',kernel_regularizer=tf.keras.regularizers.L1(l1=0),dropout=0,return_sequences=True)(input_2)
    model_2_2 = tf.keras.layers.GRU(units=512,activation='tanh',kernel_regularizer=tf.keras.regularizers.L1(l1=0),dropout=0,return_sequences=True)(model_2_2)
    model_2_2 = tf.keras.layers.Flatten()(model_2_2)
    model_2_2 = tf.keras.layers.Dense(units=n_steps_out,activation=None)(model_2_2)

    output = tf.keras.layers.Concatenate()([model_1_1, model_2_2])
    output = tf.keras.layers.Dense(units=96,activation='relu')(output)
    output = tf.keras.layers.Dense(units=96,activation='relu')(output)
    
    output = tf.keras.layers.Dense(units=n_steps_out,activation=None)(output)

    full_model = tf.keras.Model(inputs=[input_1, input_2], outputs=[output])
    
    return full_model

def build_model_sum(hourly_input_shape,daily_input_shape,n_steps_out=24):
    
    input_1 = tf.keras.layers.Input(shape=hourly_input_shape)
    input_2 = tf.keras.layers.Input(shape=daily_input_shape)

    model_1_1 = tf.keras.layers.GRU(units=448,activation='relu',kernel_regularizer=tf.keras.regularizers.L1(l1=0.06),dropout=0.36,return_sequences=True)(input_1)
    model_1_1 = tf.keras.layers.GRU(units=128,activation='relu',kernel_regularizer=tf.keras.regularizers.L1(l1=0.09),dropout=0.36,return_sequences=False)(model_1_1)
    model_1_1 = tf.keras.layers.Dense(units=n_steps_out,activation=None)(model_1_1)

    model_2_2 = tf.keras.layers.GRU(units=256,activation='relu',kernel_regularizer=tf.keras.regularizers.L1(l1=0),dropout=0,return_sequences=True)(input_2)
    model_2_2 = tf.keras.layers.GRU(units=512,activation='relu',kernel_regularizer=tf.keras.regularizers.L1(l1=0),dropout=0,return_sequences=True)(model_2_2)
    model_2_2 = tf.keras.layers.Flatten()(model_2_2)
    model_2_2 = tf.keras.layers.Dense(units=n_steps_out,activation=None)(model_2_2)

    output = tf.keras.layers.Add()([model_1_1, model_2_2])
    
    output = tf.keras.layers.Dense(units=n_steps_out,activation=None)(output)

    full_model = tf.keras.Model(inputs=[input_1, input_2], outputs=[output])
    
    return full_model

def make_predictions(model,scaler_D_x,scaler_D_y,scaler_H_x,scaler_H_y,
                     trainX_D, trainY_D, testX_D, testY_D,
                     trainX_H, trainY_H, testX_H, testY_H,
                     n_steps_out,len_output_features):
    
    # make predictions
    trainPredict = model.predict([trainX_H,trainX_D])
    trainPredict = trainPredict.reshape(trainPredict.shape[0]*n_steps_out,len_output_features)
    testPredict  = model.predict([testX_H,testX_D])
    testPredict  = testPredict.reshape(testPredict.shape[0]*n_steps_out,len_output_features)
    
    # invert predictions
    trainPredict = scaler_D_y.inverse_transform(trainPredict)
    trainY = scaler_D_y.inverse_transform(trainY_D.reshape(trainY_D.shape[0]*n_steps_out,len_output_features))
    
    testPredict = scaler_D_y.inverse_transform(testPredict)
    testY = scaler_D_y.inverse_transform(testY_D.reshape(testY_D.shape[0]*n_steps_out,len_output_features))
        
    return trainPredict,trainY,testPredict,testY

def get_metrics(trainY,trainPredict,testY,testPredict):
    
    trainMAPE  = Ep.MAPE(trainPredict,trainY)
    testMAPE  = Ep.MAPE(testPredict,testY)
    
    train_sMAPE  = Ep.sMAPE(trainY,trainPredict)
    test_sMAPE  = Ep.sMAPE(testY,testPredict)
    
    return trainMAPE,testMAPE,train_sMAPE,test_sMAPE

In [3]:
data_diaria_path = os.path.join('..','..','..','..','dataset','Series','Sabanas','Original','Sabana_Datos_Diaria.xlsx')
data_diaria = pd.read_excel(data_diaria_path)
data_diaria = data_diaria.set_index('Fecha')

In [4]:
data_horaria_path = os.path.join('..','..','..','..','dataset','Series','Sabanas','Original','Sabana_Datos_Horaria.xlsx')
data_horaria = pd.read_excel(data_horaria_path)
data_horaria = data_horaria.set_index('Fecha')

In [5]:
precio_bolsa_path = os.path.join('..','..','..','..','dataset','Series','Sabanas','Original','Sabana_Datos_Precio_Bolsa.xlsx')
precio_bolsa = pd.read_excel(precio_bolsa_path)
precio_bolsa = precio_bolsa.set_index('Fecha')

In [6]:
nombre_series_diaria = data_diaria.columns.values
nombre_series_horaria = data_horaria.columns.values

In [7]:
data_horaria_full = pd.concat([data_horaria,precio_bolsa],axis=1)

In [8]:
data_horaria.shape,data_diaria.shape, precio_bolsa.shape

((177480, 85), (7395, 119), (177480, 1))

## Build Window

In [9]:
d = 'All'
start_date_train = '2000-02-01'
start_date_val = '2020-01-01'
start_date_test = '2020-04-01'
end_date_test = '2020-05-01'
n_steps_out=24
output_columns = ['$kWh']

In [10]:
n_steps_in = 15
overlap = 1
inputs_columns = nombre_series_diaria

len_input_features = len(inputs_columns)
len_output_features = len(output_columns)

results = Ep.SplitTimeseriesMultipleTimesBackAhead_differentTimes(df_x=data_diaria,
                                                                  df_y=precio_bolsa,
                                                                  day=d,
                                                                  start_date_train=start_date_train,start_date_val=start_date_val,
                                                                  start_date_test=start_date_test,end_date_test=end_date_test,
                                                                  n_steps_out=n_steps_out,n_steps_in=n_steps_in,
                                                                  overlap=overlap,input_features=inputs_columns,
                                                                  output_features=output_columns)

trainX_D,trainY_D,valX_D,valY_D,testX_D,testY_D,scaler_D_x,scaler_D_y,dataset_x_D, dataset_y_D = results

In [11]:
n_steps_in = 360
overlap = 24
inputs_columns = nombre_series_horaria

len_input_features = len(inputs_columns)
len_output_features = len(output_columns)

results = Ep.SplitTimeseriesMultipleTimesBackAhead(df=data_horaria_full,
                                                   day=d,
                                                   start_date_train=start_date_train,start_date_val=start_date_val,
                                                   start_date_test=start_date_test,end_date_test=end_date_test,
                                                   n_steps_out=n_steps_out,n_steps_in=n_steps_in,overlap=overlap,
                                                   input_features=inputs_columns,output_features=output_columns)

trainX_H,trainY_H,valX_H,valY_H,testX_H,testY_H,scaler_H_x,scaler_H_y,dataset_x_H, dataset_y_H = results

### Train

In [12]:
'Diaria:',trainX_D.shape,trainY_D.shape,'Horaria:',trainX_H.shape, trainY_H.shape

('Diaria:',
 (7259, 15, 119),
 (7259, 24, 1),
 'Horaria:',
 (7259, 360, 85),
 (7259, 24, 1))

In [13]:
'Diaria:',valX_D.shape,valY_D.shape,'Horaria:',valX_H.shape,valY_H.shape

('Diaria:', (91, 15, 119), (91, 24, 1), 'Horaria:', (91, 360, 85), (91, 24, 1))

In [14]:
trainX_D = np.concatenate([trainX_D,valX_D])
trainY_D = np.concatenate([trainY_D,valY_D])

trainX_H = np.concatenate([trainX_H,valX_H])
trainY_H = np.concatenate([trainY_H,valY_H])

In [15]:
'Diaria:',trainX_D.shape, trainY_D.shape,'Horaria:',trainX_H.shape, trainY_H.shape

('Diaria:',
 (7350, 15, 119),
 (7350, 24, 1),
 'Horaria:',
 (7350, 360, 85),
 (7350, 24, 1))

### Test

In [16]:
'Diaria:',testX_D.shape, testY_D.shape,'Horaria:',testX_H.shape, testY_H.shape,'Imagenes:'

('Diaria:',
 (30, 15, 119),
 (30, 24, 1),
 'Horaria:',
 (30, 360, 85),
 (30, 24, 1),
 'Imagenes:')

## Model

In [17]:
log_dir = os.path.join('logs','ModeloCompleto_I5_Sum-RELU')
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
callback_tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir,
                                                          histogram_freq=2,
                                                          write_graph=True,
                                                          update_freq='epoch')

callback_reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                          factor=0.1,
                                                          min_lr=1e-5,
                                                          patience=5,
                                                          verbose=1)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  patience=10,
                                                  mode='min')

callbacks = [callback_reduce_lr,early_stopping,callback_tensorboard]

In [18]:
hourly_input_shape = (trainX_H.shape[1],trainX_H.shape[2])
daily_input_shape = (trainX_D.shape[1],trainX_D.shape[2])

model = build_model_sum(hourly_input_shape,daily_input_shape)

model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 15, 119)]    0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 360, 85)]    0                                            
__________________________________________________________________________________________________
gru_2 (GRU)                     (None, 15, 256)      289536      input_2[0][0]                    
__________________________________________________________________________________________________
gru (GRU)                       (None, 360, 448)     719040      input_1[0][0]                    
_______________________________________________________________________________________

In [19]:
model.compile(optimizer=tf.optimizers.Adam(learning_rate= 0.00257030001922535),
              loss=CustomMetrics.symmetric_mean_absolute_percentage_error,
              metrics=[tf.metrics.MeanAbsoluteError(),
                       tf.keras.metrics.MeanAbsolutePercentageError(),
                       CustomMetrics.symmetric_mean_absolute_percentage_error])

In [20]:
"""
import json

model_json = model.to_json()
with open("ModeloCompleto_I15_Sum-2.json", "w") as json_file:
    json_file.write(model_json)
"""

'\nimport json\n\nmodel_json = model.to_json()\nwith open("ModeloCompleto_I15_Sum-2.json", "w") as json_file:\n    json_file.write(model_json)\n'

In [21]:
model.fit([trainX_H,trainX_D], trainY_D, validation_data=([testX_H,testX_D],testY_D),
               epochs=200,callbacks=callbacks,verbose=1)

Epoch 1/200
  1/230 [..............................] - ETA: 0s - loss: 4505.4971 - mean_absolute_error: 0.0663 - mean_absolute_percentage_error: 273.6187 - symmetric_mean_absolute_percentage_error: 3625.7847WARNING:tensorflow:From C:\Users\SEBASTIAN\anaconda3\envs\proyecto-grados\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
230/230 [==============================] - 465s 2s/step - loss: 2976.9426 - mean_absolute_error: 3.3969 - mean_absolute_percentage_error: 25047.0449 - symmetric_mean_absolute_percentage_error: 2667.9521 - val_loss: 4987.8984 - val_mean_absolute_error: 226.9989 - val_mean_absolute_percentage_error: 290720.8125 - val_symmetric_mean_absolute_percentage_error: 4789.1230
Epoch 2/200
230/230 [==============================] - 455s 2s/step - loss: 1799.7855 - mean_absolute_error: 7.9182 -

KeyboardInterrupt: 

### Validación

In [ ]:
model.evaluate(x=[testX_H,testX_D], y=testY_D)

In [ ]:
trainPredict,trainY,testPredict,testY = make_predictions(model,scaler_D_x,scaler_D_y,scaler_H_x,scaler_H_y,
                                                         trainX_D, trainY_D, testX_D, testY_D,
                                                         trainX_H, trainY_H, testX_H, testY_H,
                                                         n_steps_out,len_output_features)

In [ ]:
trainPredict.shape,trainY.shape,testPredict.shape,testY.shape

In [ ]:
trainMAPE,testMAPE,train_sMAPE,test_sMAPE = get_metrics(trainY,trainPredict,testY,testPredict)

In [ ]:
trainMAPE,testMAPE,train_sMAPE,test_sMAPE

In [ ]:
Nt = trainPredict.shape[0] + testPredict.shape[0]
trainPredictPlot = np.zeros((Nt,1))
trainPredictPlot[:,:] = np.nan
trainPredictPlot[:len(trainPredict), :] = np.concatenate((dataset_y_D[0].reshape(1,1),trainPredict[:-1]))

# shift test predictions for plotting
testPredictPlot = np.zeros((Nt,1))
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict):Nt, :] = testPredict

plt.figure(figsize=(15,7))
plt.plot(np.concatenate((trainY,testY)),label='Original data')
plt.plot(trainPredictPlot,label='Training predictions')
plt.plot(testPredictPlot,label='Test prediction')
plt.legend()
plt.show()

In [ ]:
# shift test predictions for plotting
testOriginalPlot = np.zeros((Nt,1))
testOriginalPlot[:, :] = np.nan
testOriginalPlot[len(trainPredict):Nt, :] = testY

# shift test predictions for plotting
testPredictPlot = np.zeros((Nt,1))
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict):Nt, :] = testPredict

plt.figure(figsize=(15,7))
plt.plot(testOriginalPlot,label='Original data')
plt.plot(testPredictPlot,label='Test prediction')
plt.legend()
plt.show()